In [ ]:
import os
import time

def find_latest_directory(directory, prefix):
    # List all files/directories within the specified directory
    directories = os.listdir(directory)

    # Filter directories that start with the specified prefix
    matched_dirs = [
        d for d in directories
        if d.startswith(prefix) and os.path.isdir(os.path.join(directory, d))
    ]

    # Raise an error if no relevant directories are found
    if not matched_dirs:
        raise FileNotFoundError(
            f"No directories with prefix '{prefix}' found in the specified directory."
        )

    # Find the latest directory by extracting and comparing the timestamp part of the directory name
    latest_dir = max(matched_dirs, key=lambda d: int(d.split("_")[-1]))

    # Join the specified directory path with the latest directory name
    latest_dir_path = os.path.join(directory, latest_dir)

    return latest_dir_path

home_directory = os.path.expanduser("~")  # Get the home directory
cgmres_directory_path = os.path.join(
    home_directory, ".ros/log"
)  # Set the directory path

latest_cgmres_directory = find_latest_directory(
    cgmres_directory_path, prefix="cgmres_debug_"
)
latest_trajectory_directory = find_latest_directory(
    cgmres_directory_path, prefix="trajectory_"
)

# Get the creation time of the latest_cgmres_directory
cgmres_creation_time = os.path.getmtime(latest_cgmres_directory)
cgmres_formatted_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(cgmres_creation_time))

# Get the creation time of the latest_trajectory_directory
trajectory_creation_time = os.path.getmtime(latest_trajectory_directory)
trajectory_formatted_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(trajectory_creation_time))

print(f"The latest cgmres debug directory is: {latest_cgmres_directory}")
print(f"The latest cgmres debug directory was created at: {cgmres_formatted_time}")

print(f"The latest trajectory directory is: {latest_trajectory_directory}")
print(f"The latest trajectory directory was created at: {trajectory_formatted_time}")

In [ ]:
from plotter import Plotter

plotter = Plotter(log_dir=latest_cgmres_directory)
plotter.set_scales(2,5,2)
plotter.show()
# plotter.save()

In [ ]:
import importlib
import animator  
importlib.reload(animator)
from animator import TrajectoryFollowing


anim = TrajectoryFollowing(
    cgmres_log_dir=latest_cgmres_directory,
    trajectory_log_dir=latest_trajectory_directory
)
# anim.set_skip_frames(10)
anim.generate_animation()


In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

plots_dir = os.path.join(latest_cgmres_directory, "plots")

# 画像ファイルのリストを取得し、自然順にソート
image_files = sorted([f for f in os.listdir(plots_dir) if f.endswith(".png")])

# 最初と最後の画像のインデックスを取得
first_index = 0
last_index = len(image_files) - 1

# 最初と最後の画像以外の8枚の画像のインデックスを計算
num_images = 10
indices = np.linspace(first_index, last_index, num_images, dtype=int)

# 選択された画像のファイル名を取得
selected_images = [image_files[i] for i in indices]

# 画像を読み込んでサブプロットに表示
fig, axes = plt.subplots(5, 2, figsize=(10, 20))
axes = axes.ravel()

for i, image_file in enumerate(selected_images):
    img = plt.imread(os.path.join(plots_dir, image_file))
    axes[i].imshow(img)
    axes[i].axis("off")
    axes[i].set_title(f"Frame {os.path.splitext(image_file)[0][6:]}")

# レイアウトを調整
plt.tight_layout()

# 画像を表示
plt.show()

In [ ]:
import os
import sys

relative_path = "../../mpc_lateral_controller/debug"
absolute_path = os.path.abspath(relative_path)

if os.path.exists(absolute_path):
    sys.path.append(absolute_path)
else:
    print(f"The path {absolute_path} does not exist.")

import importlib
import visualize_path

importlib.reload(visualize_path)
from visualize_path import plot_trajectory

# データディレクトリを指定する場合は、以下のコメントアウトを解除し、適切なパスを設定してください。
# data_directory = "/path/to/your/data/directory"
# plot_trajectory(data_directory)

# デフォルトの最新ディレクトリを使用する場合は、引数なしで呼び出します。
plot_trajectory()

# ウィンドウで表示したい場合は、以下のコードを実行してください。
# python3 src/autoware/universe/control/mpc_lateral_controller/debug/visualize_path.py